## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
from sklearn.model_selection import train_test_split , KFold,GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
from sklearn import datasets,metrics


C:\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [7]:
## 讀取資料
wins = datasets.load_wine()

### 切分訓練集/測試集
x_train,x_test,y_train,y_test = train_test_split(wins.data,wins.target,test_size=0.3,random_state=42)

## 建立模型
clf = GradientBoostingClassifier(random_state=7)

In [11]:
# 先看看使用預設參數得到的結果，準確率約為 0.94444
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(metrics.accuracy_score(y_test,y_pred))

0.9444444444444444


In [13]:
# 設定要訓練的超參數組合
n_estimators = [150, 200, 300]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    1.0s finished


In [14]:
 # 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -0.040323 using {'max_depth': 1, 'n_estimators': 200}


In [15]:
grid_result.best_params_

{'max_depth': 1, 'n_estimators': 200}

In [17]:
# 使用最佳參數重新建立模型
clf_bestpram = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'],
                                         n_estimators = grid_result.best_params_['n_estimators'])
# 訓練模型
clf_bestpram.fit(x_train,y_train)

# 預測測試集
y_pred = clf_bestpram.predict(x_test)

In [18]:
# 調整參數後 準確率提升至1
print(metrics.accuracy_score(y_test,y_pred))

1.0
